<h1><font size="8">SVM (Support Vector Machines)</font></h1>

Created by: M. Ben-Azzouz (moez.ben-azzouz@sinclair.edu)

# Part I - Background

<p>In this notebook, you will use SVM (Support Vector Machines) to build and train a model using human cell records, and classify cells to whether the samples are benign or malignant.</p>

<p>SVM works by mapping data to a high-dimensional feature space so that data points can be categorized, even when the data are not otherwise linearly separable. A separator between the categories is found, then the data is transformed in such a way that the separator could be drawn as a hyperplane. Following this, characteristics of new data can be used to predict the group to which a new record should belong.</p>

## Load Relevant Libraries and Packages

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

## Load the Cancer data

<p>The example is based on a dataset that is publicly available from the UCI Machine Learning Repository (Asuncion and Newman, 2007)[http://mlearn.ics.uci.edu/MLRepository.html]. The dataset consists of several hundred human cell sample records, each of which contains the values of a set of cell characteristics. The fields in each record are:</p>

|Field name|Description|
|--- |--- |
|ID|Clump thickness|
|Clump|Clump thickness|
|UnifSize|Uniformity of cell size|
|UnifShape|Uniformity of cell shape|
|MargAdh|Marginal adhesion|
|SingEpiSize|Single epithelial cell size|
|BareNuc|Bare nuclei|
|BlandChrom|Bland chromatin|
|NormNucl|Normal nucleoli|
|Mit|Mitoses|
|Class|Benign or malignant|

<br>
<br>

<p>For the purposes of this example, we're using a dataset that has a relatively small number of predictors in each record. The data we will use in in the 'cell_samples.csv' file. Be sure that the file in in the same directory as this notebook before loading it in the following cell.</p>

## Load Data From CSV File

<p>Load the data in the 'cell_samples.csv' into a Pandas dataframe. Pandas is a rich Python library for manipulating data. It is used extensively in data science and machine learning. You are encouraged to learn more about Pandas at https://pandas.pydata.org/docs/getting_started/index.html</p>

In [ ]:
cell_df = pd.read_csv("cell_samples.csv")
cell_df.head()

<p>The ID field contains the patient identifiers. The characteristics of the cell samples from each patient are contained in fields Clump to Mit. The values are graded from 1 to 10, with 1 being the closest to benign.</p>
<p>The Class field contains the diagnosis, as confirmed by separate medical procedures, as to whether the samples are benign (value = 2) or malignant (value = 4).</p>
<p>Lets look at the distribution of the classes based on Clump thickness and Uniformity of cell size:</p>

In [ ]:
ax = cell_df[cell_df['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='malignant');
cell_df[cell_df['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='benign', ax=ax);
plt.show()

## Data Pre-Processing and Selection

<p>One of the most important steps when building models in data science and machine learning is the data pre-processing step. Typically this involves dealing with missing or invalid data points, normalizing and/or standardizing numeric data, combining features, and converting categorical data to numeric data. In this example, the data we are starting with has been most cleaned up, but we will need to tweak it a bit further before we train an SVM model based on this data. However, typically when working on real-world problems, data scientists and machine learning engineers face unstructured data that may come from different sources and have gaps or missing values. So, the pre-processing task would require a lot more work and care.</p>
<p>Lets first look at columns data types:</p>

In [ ]:
cell_df.dtypes     # The dtypes attribute reveal the data type of each feature (variable). Object means nonnumeric data.

It looks like the __BareNuc__ column includes some values that are not numerical. We can drop those rows:

In [ ]:
cell_df = cell_df[pd.to_numeric(cell_df['BareNuc'], errors='coerce').notnull()]
cell_df['BareNuc'] = cell_df['BareNuc'].astype('int')
cell_df.dtypes

In [ ]:
feature_df = cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]

We want the model to predict the value of Class (that is, benign (=2) or malignant (=4)). As this field can have one of only two possible values, we need to change its measurement level to reflect this.

In [ ]:
cell_df['Class'] = cell_df['Class'].astype('int')
y = np.asarray(cell_df['Class'])
y [0:5]

## Train/Test dataset

We now split our dataset into train and test sets (we will use 20% of the data for testing and 80% for training):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

## Modeling (SVM with Scikit-Learn)

The SVM algorithm offers a choice of kernel functions for performing its processing. Basically, mapping data into a higher dimensional space is called kernelling. The mathematical function used for the transformation is known as the kernel function, and can be of different types, such as:

    1.Linear
    2.Polynomial
    3.Radial basis function (RBF)
    4.Sigmoid
Each of these functions has its characteristics, its pros and cons, and its equation, but as there's no easy way of knowing which function performs best with any given dataset, we usually choose different functions in turn and compare the results. Let's first use the default RBF(Radial Basis Function) for this lab.

In [ ]:
from sklearn import svm                        # Import the svm module from sci-kit learn
clf = svm.SVC(kernel='rbf', gamma='auto')      # Run the SVM constructor function SVC() and store the resulting object in clf
clf.fit(X_train, y_train)                      # Fit the model to the training data

After being fitted, the model can then be used to predict new values:

In [ ]:
yhat = clf.predict(X_test)                   # Use the model to predict the results of the test data and store in y-hat
yhat [0:5]                                   # Display a sample from y-hat (the first five entries)

## Evaluation

How do we decide whether our model's predicitions are acceptable? We need ways to assess our results. We will use some evaluation metrics. There are various types of metrics at our disposal, each type has its pros and cons and picking the right one will typically depend on the model and data we are evaluating. It is possible for several metrics to be appropriate for the same scenario. We will demonstrate a few metrics here.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

### Confusion Matrix

A confusion matrix is a simple way of understanding how well an algorithm is doing in classifying data. It is really just the idea of false positives and false negatives. A false positive (FP) means the input data point is classified as positive (e.g. tissue is cancerous) by our model but it is in fact negative (non-cancerous tissue cell). A false negative (FN) means the tested data point is classified as negative (non-cancerous) while in reality it should be positive (cancerous). Finally, as you may have guessed, true positive (TP) and true negative (TN) mean that the model correctly classified the data as positive or negative, respectively. The matrix displays these four numbers. Based on the information in a confusion matrix we can further compute the following metrics:

Accuracy rate: (TP + TN)/Total, where total = TP + TN + FP + FN.

Error rate: 1- Accuracy rate

True positive rate (also called Recall): TP/(TP + FN)

False positive rate: FP/(TN + FP)

True negative rate (also called Specificity): 1 - false positive rate

Precision: TP/(TP + FP). This rates how often algorithm gets it correct with true positives.

Prevalence: (TP + FN)/Total.

Which of these measures is best depends on the context. For example, in our case where we are testing for cancer, then we would want a small false negative rate and aren't too concerned about high false positives. On the other hand, we don't want to go around predicting everyone has cancer.

We have created a Python function below that will be used to plot the confusion matrix.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

We will now create the confusion matrix for our model and compute the metrics discussed above (using classification_report from the sklearn library) and plot the matrix using our custom plotting function.

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[2,4])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Benign(2)','Malignant(4)'],normalize= False,  title='Confusion matrix')

Observe above that TP = 47, TN = 85, FP = 5, and FN = 0. If we don't want to see all the metrics above, we can compute specific scores from sklearn library (e.g. f1 score and Jaccard similarity score below). 

You can also easily use the __f1_score__ from sklearn library:

You can learn more about this metric at https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

In [ ]:
from sklearn.metrics import f1_score        # F1-Score can be interpreted as a weighted avg of the precision and recall
f1_score(y_test, yhat, average='weighted')  # It's a number between 0 and 1, where 1 is the best score

Lets try jaccard index for accuracy:

You can learn more about this metric at https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html#sklearn.metrics.jaccard_score

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat, average='weighted')

# Part II - Assignment

## Write you name in the markdown cell below

1) Rebuild the model usinga 'linear' kernel (use the __kernel='linear'__ option), and recompute the f1 and Jaccard scores.

In [ ]:
# Train the model in this cell (note: you do not need to re-import the libraries imported in Part I)


In [ ]:
# Use the model to make predictions using the test data in this cell


In [ ]:
# Evaluate the model in this cell


2) Rebuild the model usinga 'sigmoid' kernel (use the __kernel='sigmoid'__ option), and recompute the f1 and Jaccard scores.

In [ ]:
# Train the model in this cell


In [ ]:
# Use the model to make predictions using the test data in this cell


In [ ]:
# Evaluate the model in this cell


3) Rebuild the model usinga 'polynomial' kernel (use the __kernel='polynomial'__ option), and recompute the f1 and Jaccard scores.

In [ ]:
# Train the model in this cell


In [ ]:
# Use the model to make predictions using the test data in this cell


In [ ]:
# Evaluate the model in this cell


4) Compare the results from all four kernels and write a conclusion (in a Markdown cell below) where you will discuss the best choice and why.